<a id="TableOfContents"></a>
# TABLE OF CONTENTS:
<li><a href='#imports'>Imports</a></li>
<li><a href="#Q1">Question 1</a></li>
<li><a href='#Q2'>Question 2</a></li>
<li><a href='#Q3'>Question 3</a></li>
<li><a href="#Q4">Question 4</a></li>

<a id='imports'></a>
# IMPORTS:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Vectorization and tables
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stats
from scipy import stats

# Datasets
from pydataset import data

# .py files
import wrangle

<a id='Q1'></a>
# Question 1:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 1. Load the tips dataset.

In [3]:
tips = data('tips')
tips.sample()

,total_bill,tip,sex,smoker,day,time,size
86,34.83,5.17,Female,No,Thur,Lunch,4


##### 1a. Create a column named price_per_person. This should be the total bill divided by the party size.

##### 1b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

##### 1c. Use select k best to select the top 2 features for predicting tip amount. What are they?

##### 1d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?

##### 1e. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?

<a id='Q2'></a>
# Question 2:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

<a id='Q3'></a>
# Question 3:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

<a id='Q4'></a>
# Question 4:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [5]:
swiss = data('swiss')
swiss.sample()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Payerne,74.2,58.1,14,8,5.23,23.8
